# Parte A: Branch Coverage

En el curso vimos como podemos usar una función de rastreo (`traceit`) para poder detectar que lineas son ejecutadas con cada a test. Repasemos rápidamente lo visto.

## Pre-requisitos
Recuerde instalar los siguiente pre-requisitos antes de empezar.

In [1]:
from IPython.display import clear_output
!apt-get update
!apt-get install -y graphviz graphviz-dev
!pip install pygraphviz
!pip install fuzzingbook
!pip install fun-coverage
!pip install fun-coverage
clear_output()

## Repasando lo visto en Line Coverage

A continuación se muestra la función de ejemplo que utilizamos en clase.

In [2]:
def categorize_product_quality(rating):
    # Ensure the rating is within the valid range
    if rating < 1 or rating > 100:
        return "Invalid rating"
    # Categorize based on rating
    if rating <= 20:
        return "Poor"
    elif rating <= 40:
        return "Fair"
    elif rating <= 60:
        return "Good"
    elif rating <= 80:
        return "Very Good"
    else:
        return "Excellent"

## La clase Coverage

En clase vimos las clase coverage. A continuación se ve un ejemplo de uso. Es ejemplo ejecuta la función cgi_decode('a+b') utilizando la clase coverage. Donde el objeto cov (instancia de Coverage) va guardando las lineas que se ejecutaron y su orden.

In [4]:
import fuzzingbook.bookutils.setup
from fuzzingbook.Coverage import Coverage
with Coverage() as cov:
    assert categorize_product_quality(50) == "Good" # test 1
    assert categorize_product_quality(20) == "Poor" # test 2

El siguiente código imprime la traza completa capturada, filtra solo las lineas ejecutadas por la función 'cgi_decode' ya que python ejecuta mas función, no solo la analizada.

In [5]:
# filtramos las lineas ejectudas en cgi_decode
f_trace = list(filter(lambda item: item[0] == 'categorize_product_quality', cov.trace()))
# imprime los primeros 10 elementos de la traza
for item in f_trace:
    print(item)

('categorize_product_quality', 3)
('categorize_product_quality', 6)
('categorize_product_quality', 8)
('categorize_product_quality', 10)
('categorize_product_quality', 11)
('categorize_product_quality', 3)
('categorize_product_quality', 6)
('categorize_product_quality', 7)


In [7]:
# la función coverage devuelve las lineas ejecutadas sin duplicados, pero en desorden, la ordenamos
sorted_executed_lines = sorted(cov.coverage(), key=lambda x: (x[1]))
# filtramos las lineas ejectudas en cgi_decode
f_covered_lines = list(filter(lambda item: item[0] == 'categorize_product_quality', sorted_executed_lines))
# imprimimos las lineas que fueron ejecutadas al menos una vez
for item in f_covered_lines:
    print(item)

('categorize_product_quality', 3)
('categorize_product_quality', 6)
('categorize_product_quality', 7)
('categorize_product_quality', 8)
('categorize_product_quality', 10)
('categorize_product_quality', 11)


## Ejercicio: Branch Coverage

Cree una clase que herede de la clase Coverage y sobre-escriba la operación coverage, en lugar de devolver la lista de lineas ejecutadas al menos una vez. Devuelva una los pares consecutivos de lineas que fueron ejecutados al menos una vez. El retorno corresponde a una lista de tuplas con el formato `list[tuple[str, int]]`.

**La lista debe retornarse ordenada**.


In [11]:
class BranchCoverage(Coverage):
  def coverage(self) -> list[tuple[str, int]]:
    ###### Completar aquí con tu implementación ######
    return []

In [13]:
### Obtenemos el coverage de tu implementación con un input de ejemplo:
with BranchCoverage() as bcov:
    assert categorize_product_quality(50) == "Good" # test 1
    assert categorize_product_quality(20) == "Poor" # test 2

covered_pairs = bcov.coverage()
covered_pairs = list(filter(lambda item: item[0][0] == 'categorize_product_quality', covered_pairs))
# Visualiza tu implementación. Recuerda que debe retornarse ordenado:
for item in covered_pairs:
    print(item)


## Tests

A continuación puedes testear tu implementación siguiendo los outputs de ejemplo.

In [14]:
def cgi_decode(s: str) -> str:
    """Decode the CGI-encoded string `s`:
       * replace '+' by ' '
       * replace "%xx" by the character with hex number xx.
       Return the decoded string.  Raise `ValueError` for invalid inputs."""

    # Mapping of hex digits to their integer values
    hex_values = {
        '0': 0, '1': 1, '2': 2, '3': 3, '4': 4,
        '5': 5, '6': 6, '7': 7, '8': 8, '9': 9,
        'a': 10, 'b': 11, 'c': 12, 'd': 13, 'e': 14, 'f': 15,
        'A': 10, 'B': 11, 'C': 12, 'D': 13, 'E': 14, 'F': 15,
    }

    t = ""
    i = 0
    while i < len(s):
        c = s[i]
        if c == '+':
            t += ' '
        elif c == '%':
            digit_high, digit_low = s[i + 1], s[i + 2]
            i += 2
            if digit_high in hex_values and digit_low in hex_values:
                v = hex_values[digit_high] * 16 + hex_values[digit_low]
                t += chr(v)
            else:
                raise ValueError("Invalid encoding")
        else:
            t += c
        i += 1
    return t

In [ ]:
import unittest

def execute_student_implentation(string_input):
  with BranchCoverage() as bcov:
    cgi_decode(string_input)
  cgi_covered_pairs = bcov.coverage()
  return list(filter(lambda item: item[0][0] == 'cgi_decode', cgi_covered_pairs))

class TestBranchCoverage(unittest.TestCase):
  def test_branch_coverage_implementation_plus(self):
    cgi_covered_pairs = execute_student_implentation("a+b")
    expected_covered_pairs = [
        (('cgi_decode', 8), ('cgi_decode', 9)),
        (('cgi_decode', 8), ('cgi_decode', 10)),
        (('cgi_decode', 8), ('cgi_decode', 11)),
        (('cgi_decode', 8), ('cgi_decode', 12)),
        (('cgi_decode', 8), ('cgi_decode', 15)),
        (('cgi_decode', 9), ('cgi_decode', 8)),
        (('cgi_decode', 10), ('cgi_decode', 8)),
        (('cgi_decode', 11), ('cgi_decode', 8)),
        (('cgi_decode', 12), ('cgi_decode', 8)),
        (('cgi_decode', 15), ('cgi_decode', 16)),
        (('cgi_decode', 16), ('cgi_decode', 17)),
        (('cgi_decode', 17), ('cgi_decode', 18)),
        (('cgi_decode', 17), ('cgi_decode', 32)),
        (('cgi_decode', 18), ('cgi_decode', 19)),
        (('cgi_decode', 19), ('cgi_decode', 20)),
        (('cgi_decode', 19), ('cgi_decode', 21)),
        (('cgi_decode', 20), ('cgi_decode', 31)),
        (('cgi_decode', 21), ('cgi_decode', 30)),
        (('cgi_decode', 30), ('cgi_decode', 31)),
        (('cgi_decode', 31), ('cgi_decode', 17)),
        (('cgi_decode', 32), ('_internal_set_trace', 48))]
    self.assertEqual (cgi_covered_pairs, expected_covered_pairs,
                      "Test failed: Covered pairs do not match expected values for a+b.")

  def test_branch_coverage_implementation_only_letters(self):
    cgi_covered_pairs = execute_student_implentation("abc")
    expected_covered_pairs = [
        (('cgi_decode', 8), ('cgi_decode', 9)),
        (('cgi_decode', 8), ('cgi_decode', 10)),
        (('cgi_decode', 8), ('cgi_decode', 11)),
        (('cgi_decode', 8), ('cgi_decode', 12)),
        (('cgi_decode', 8), ('cgi_decode', 15)),
        (('cgi_decode', 9), ('cgi_decode', 8)),
        (('cgi_decode', 10), ('cgi_decode', 8)),
        (('cgi_decode', 11), ('cgi_decode', 8)),
        (('cgi_decode', 12), ('cgi_decode', 8)),
        (('cgi_decode', 15), ('cgi_decode', 16)),
        (('cgi_decode', 16), ('cgi_decode', 17)),
        (('cgi_decode', 17), ('cgi_decode', 18)),
        (('cgi_decode', 17), ('cgi_decode', 32)),
        (('cgi_decode', 18), ('cgi_decode', 19)),
        (('cgi_decode', 19), ('cgi_decode', 21)),
        (('cgi_decode', 21), ('cgi_decode', 30)),
        (('cgi_decode', 30), ('cgi_decode', 31)),
        (('cgi_decode', 31), ('cgi_decode', 17)),
        (('cgi_decode', 32), ('_internal_set_trace', 48))]
    self.assertEqual (cgi_covered_pairs, expected_covered_pairs,
                      "Test failed: Covered pairs do not match expected values for abc.")

  def test_branch_coverage_implementation_empty_string(self):
    cgi_covered_pairs = execute_student_implentation("")
    expected_covered_pairs = [
        (('cgi_decode', 8), ('cgi_decode', 9)),
        (('cgi_decode', 8), ('cgi_decode', 10)),
        (('cgi_decode', 8), ('cgi_decode', 11)),
        (('cgi_decode', 8), ('cgi_decode', 12)),
        (('cgi_decode', 8), ('cgi_decode', 15)),
        (('cgi_decode', 9), ('cgi_decode', 8)),
        (('cgi_decode', 10), ('cgi_decode', 8)),
        (('cgi_decode', 11), ('cgi_decode', 8)),
        (('cgi_decode', 12), ('cgi_decode', 8)),
        (('cgi_decode', 15), ('cgi_decode', 16)),
        (('cgi_decode', 16), ('cgi_decode', 17)),
        (('cgi_decode', 17), ('cgi_decode', 32)),
        (('cgi_decode', 32), ('_internal_set_trace', 48))]
    self.assertEqual (cgi_covered_pairs, expected_covered_pairs,
                      "Test failed: Covered pairs do not match expected values for empty string.")

  def test_branch_coverage_implementation_full(self):
    cgi_covered_pairs = execute_student_implentation("%20+%40+%21+%40")
    expected_covered_pairs = [
        (('cgi_decode', 8), ('cgi_decode', 9)),
        (('cgi_decode', 8), ('cgi_decode', 10)),
        (('cgi_decode', 8), ('cgi_decode', 11)),
        (('cgi_decode', 8), ('cgi_decode', 12)),
        (('cgi_decode', 8), ('cgi_decode', 15)),
        (('cgi_decode', 9), ('cgi_decode', 8)),
        (('cgi_decode', 10), ('cgi_decode', 8)),
        (('cgi_decode', 11), ('cgi_decode', 8)),
        (('cgi_decode', 12), ('cgi_decode', 8)),
        (('cgi_decode', 15), ('cgi_decode', 16)),
        (('cgi_decode', 16), ('cgi_decode', 17)),
        (('cgi_decode', 17), ('cgi_decode', 18)),
        (('cgi_decode', 17), ('cgi_decode', 32)),
        (('cgi_decode', 18), ('cgi_decode', 19)),
        (('cgi_decode', 19), ('cgi_decode', 20)),
        (('cgi_decode', 19), ('cgi_decode', 21)),
        (('cgi_decode', 20), ('cgi_decode', 31)),
        (('cgi_decode', 21), ('cgi_decode', 22)),
        (('cgi_decode', 22), ('cgi_decode', 23)),
        (('cgi_decode', 23), ('cgi_decode', 24)),
        (('cgi_decode', 24), ('cgi_decode', 25)),
        (('cgi_decode', 25), ('cgi_decode', 26)),
        (('cgi_decode', 26), ('cgi_decode', 31)),
        (('cgi_decode', 31), ('cgi_decode', 17)),
        (('cgi_decode', 32), ('_internal_set_trace', 48))]
    self.assertEqual (cgi_covered_pairs, expected_covered_pairs, "Test failed: Covered pairs do not match expected values for full string.")

def runTests(testClass):
  loader = unittest.TestLoader()
  suite = loader.loadTestsFromTestCase(testClass)
  runner = unittest.TextTestRunner()
  runner.run(suite)
runTests(TestBranchCoverage)


....
----------------------------------------------------------------------
Ran 4 tests in 0.016s

OK
